In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Obstructive_sleep_apnea/GSE168358'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Pathochip data of tonsil and lymph node samples"
!Series_summary	"The Microbiome of HPV-Positive Tonsil Squamous Cell Carcinoma and Neck Metastasis"
!Series_overall_design	"Analysis of the different microbial species detected in the primary tumor specimens and lymph nodes"
Sample Characteristics Dictionary:
{0: ['sample type: RNA and DNA']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine gene expression data availability
if "rna and dna" in [val.lower() for sublist in sample_characteristics_dict.values() for val in sublist]:
    is_gene_available = True

# Variable availability analysis
for key, values in sample_characteristics_dict.items():
    for value in values:
        if 'obstructive_sleep_apnea' in value.lower():
            trait_row = key
        if 'age' in value.lower():
            age_row = key
        if 'gender' in value.lower():
            gender_row = key

# Define functions to convert data types
def convert_trait(value):
    value = value.split(':')[-1].strip().lower()
    if value == 'positive':
        return 1
    elif value == 'negative':
        return 0
    return None

def convert_age(value):
    try:
        return float(value.split(':')[-1].strip())
    except ValueError:
        return None

def convert_gender(value):
    gender = value.split(':')[-1].strip().lower()
    if gender == 'female':
        return 0
    elif gender == 'male':
        return 1
    return None

# Save cohort information
save_cohort_info('GSE168358', './preprocessed/Obstructive_sleep_apnea/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Obstructive_sleep_apnea', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Obstructive_sleep_apnea/trait_data/GSE168358.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
